In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  9 16:47:45 2024

@author: bingh
"""

import pandas as pd
import folium
from folium.plugins import MeasureControl
from geopy.geocoders import Nominatim
import time
import re
import os
import random
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# ------------------------
# Load input settings
# ------------------------
InputFilePath = os.getcwd() + '/'
Input_setting = pd.read_excel(os.path.join(InputFilePath, 'Visualisation_Input_Setting.xlsx'), index_col='Setting')

address_file = Input_setting['Path']['address_file']
cache_file = Input_setting['Path']['cache_file']
output_map = Input_setting['Path']['output_map']
failed_output_file = Input_setting['Path']['failed_output_file']

def super_clean_address(address):
    address = address.replace('BLK', '').replace('BIK', '').replace('BKL', '')
    address = re.sub(r'#\S+', '', address)
    address = re.sub(r'-\d+\s', '', address)
    address = re.sub(r'\(.*?\)', '', address)
    address = re.sub(r'Level\s*\d+', '', address, flags=re.IGNORECASE)
    address = re.sub(r'\b(L\d|B\d|#\d{1,3})\b', '', address, flags=re.IGNORECASE)
    address = re.sub(r'\s+', ' ', address).strip()

    postal_search = re.search(r'(S\d{6})', address)
    if postal_search:
        postal_code = postal_search.group(1)
        address_parts = address.split(postal_code)
        address_cleaned = address_parts[0].strip() + " " + postal_code
    else:
        address_cleaned = address.strip()

    return address_cleaned

def geocode_address(geolocator, address):
    try:
        location = geolocator.geocode(address + ", Singapore")
        if location:
            return location.latitude, location.longitude, location.address
        else:
            postal_search = re.search(r'(S\d{6})', address)
            if postal_search:
                postal_code = postal_search.group(1)
                location = geolocator.geocode(postal_code + ", Singapore")
                if location:
                    return location.latitude, location.longitude, location.address
        return None, None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None, None

def load_paths_from_excel(setting_file):
    df = pd.read_excel(setting_file)
    path_dict = {}
    for idx, row in df.iterrows():
        key = row['Setting']
        path = row['Path'].replace('/', '\\')  # Auto correct slash
        path_dict[key] = path
    return path_dict

def plot_addresses_final(setting_file):
    paths = load_paths_from_excel(setting_file)

    address_file = paths['address_file']
    cache_file = paths['cache_file']
    output_map = paths['output_map']
    failed_output_file = paths['failed_output_file']

    if not os.path.exists(address_file):
        print(f"Address file not found: {address_file}")
        return

    df = pd.read_csv(address_file)

    if not {'Outlet Address', 'Staff'}.issubset(df.columns):
        print("Input must have 'Outlet Address' and 'Staff' columns!")
        return

    if os.path.exists(cache_file):
        cache_df = pd.read_csv(cache_file)
    else:
        cache_df = pd.DataFrame(columns=['Original Address', 'Cleaned Address', 'Latitude', 'Longitude', 'Full Address', 'Staff'])

    geolocator = Nominatim(user_agent="singapore_final_mapper")

    map_sg = folium.Map(location=[1.3521, 103.8198], zoom_start=11)
    map_sg.add_child(MeasureControl(primary_length_unit='kilometers', secondary_length_unit='meters'))

    color_palette = [
        'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
        'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',
         'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'
    ]
    random.shuffle(color_palette)
    merchandiser_color = {}

    failed_list = []
    staff_groups = {}  

    for idx, row in df.iterrows():
        original_address = row['Outlet Address']
        merchandiser = row['Staff']

        cleaned_address = super_clean_address(original_address)

        cached_row = cache_df[cache_df['Original Address'] == original_address]
        if not cached_row.empty:
            lat = cached_row.iloc[0]['Latitude']
            lon = cached_row.iloc[0]['Longitude']
            full_address = cached_row.iloc[0]['Full Address']
            print(f"Using cached: {cleaned_address}")
        else:
            lat, lon, full_address = geocode_address(geolocator, cleaned_address)
            if lat and lon:
                new_row = pd.DataFrame({
                    'Original Address': [original_address],
                    'Cleaned Address': [cleaned_address],
                    'Latitude': [lat],
                    'Longitude': [lon],
                    'Full Address': [full_address],
                    'Staff': [merchandiser]
                })
                cache_df = pd.concat([cache_df, new_row], ignore_index=True)
                print(f"Plotted new: {cleaned_address}")
            else:
                print(f"Failed to geocode: {cleaned_address}")
                failed_list.append({'Outlet Address': original_address, 'Cleaned Address': cleaned_address, 'Staff': merchandiser})
                continue

            time.sleep(1)

        if lat < 1.22 or lat > 1.47 or lon < 103.6 or lon > 104.1:
            print(f"Skipping: {full_address}")
            failed_list.append({'Outlet Address': original_address, 'Cleaned Address': cleaned_address, 'Staff': merchandiser})
            continue

        if merchandiser not in merchandiser_color:
            merchandiser_color[merchandiser] = color_palette[len(merchandiser_color) % len(color_palette)]

        if merchandiser not in staff_groups:
            staff_groups[merchandiser] = folium.FeatureGroup(name=merchandiser, show=True)

        popup_html = f"""
        <b>Outlet Address:</b> {full_address}<br>
        <b>Merchandiser (Staff):</b> {merchandiser}
        """
        marker = folium.Marker(
            [lat, lon],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=merchandiser,
            icon=folium.Icon(color=merchandiser_color[merchandiser], icon="info-sign")
        )

        marker.add_to(staff_groups[merchandiser])

    # Add all staff layers
    for staff_layer in staff_groups.values():
        staff_layer.add_to(map_sg)

    folium.LayerControl(collapsed=False).add_to(map_sg)

    cache_df.to_csv(cache_file, index=False)
    print(f"\nCache updated and saved to: {cache_file}")

    if failed_list:
        failed_df = pd.DataFrame(failed_list)
        failed_df.to_csv(failed_output_file, index=False)
        print(f"Failed addresses saved to: {failed_output_file}")
    else:
        print("No failed addresses!")

    map_sg.save(output_map)
    print(f"\nMap successfully saved to: {output_map}")

if __name__ == "__main__":
    setting_file = os.path.join(os.getcwd(), "Visualisation_Input_Setting.xlsx")
    plot_addresses_final(setting_file)
